In [5]:
import pandas as pd
import glob
import os
from pprint import pprint

In [9]:
#Get all CSV files in the folder of GA
all_files = glob.glob(r"C:\Huy Phan\College\VoterTurnout\data\NM\*.csv")

# Files that contain both 'precinct' and 'general' in the filename
general_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'general'])
]


# Files that contain both 'precinct' and 'primary' in the filename
primary_files = [
    f for f in all_files
    if all(word in f.lower() for word in ['precinct', 'primary'])
]


In [7]:
print("General files:")
for f in general_files:
    print(f)


General files:
C:\Huy Phan\College\VoterTurnout\data\NM\nm__general__precinct.csv


In [10]:
print("\nPrimary files:")
for f in primary_files:
    print(f)


Primary files:
C:\Huy Phan\College\VoterTurnout\data\NM\nm__primary__precinct.csv


In [14]:
# Process primary files
primary_df_list = []

for file in primary_files:
    try:
        df = pd.read_csv(file)

        # Drop rows where 'precinct' is NaN
        if 'precinct' in df.columns:
            df = df[df["precinct"].notna()]
        
        if 'office' in df.columns:
            df = df[df["office"] == "President"]

        df = df.drop_duplicates()

        primary_df_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
pri_combined_df = pd.concat(primary_df_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
pri_combined_df["precinct"] = pri_combined_df["county"].astype(str) + pri_combined_df["precinct"].astype(str)


In [15]:
pri_combined_df

,county,precinct,office,district,party,candidate,votes
0,Bernalillo,Bernalillo1,President,NaN,Democratic,HILLARY RODHAM CLINTON,60
1,Bernalillo,Bernalillo1,President,NaN,Democratic,BERNARD SANDERS,93
2,Bernalillo,Bernalillo1,President,NaN,Republican,DONALD J TRUMP,64
3,Bernalillo,Bernalillo1,President,NaN,Republican,JEB BUSH,3
4,Bernalillo,Bernalillo1,President,NaN,Republican,BENJAMIN S CARSON,6
...,...,...,...,...,...,...,...
11931,Valencia,Valencia41,President,NaN,Republican,JEB BUSH,1
11932,Valencia,Valencia41,President,NaN,Republican,BENJAMIN S CARSON,1
11933,Valencia,Valencia41,President,NaN,Republican,TED CRUZ,25
11934,Valencia,Valencia41,President,NaN,Republican,CARLY FIORINA,0


In [44]:
# Select only the relevant columns
primary_data = pri_combined_df[["precinct", "party", "candidate", "votes"]]
primary_data["party"].value_counts(dropna=False)

party
Republican    8952
Democratic    2984
Name: count, dtype: int64

In [45]:
primary_data.loc[:, "party"] = primary_data["party"].replace({
    "Democratic": "DEM",
    "Republican": "REP"
})

# primary_data = primary_data[~primary_data["candidate"].isin(["YES", "NO"])]
# primary_data = primary_data[primary_data["party"].isin(["DEM", "REP"])] # Analyzing only republican and democratic

primary_data

,precinct,party,candidate,votes
0,Bernalillo1,DEM,HILLARY RODHAM CLINTON,60
1,Bernalillo1,DEM,BERNARD SANDERS,93
2,Bernalillo1,REP,DONALD J TRUMP,64
3,Bernalillo1,REP,JEB BUSH,3
4,Bernalillo1,REP,BENJAMIN S CARSON,6
...,...,...,...,...
11931,Valencia41,REP,JEB BUSH,1
11932,Valencia41,REP,BENJAMIN S CARSON,1
11933,Valencia41,REP,TED CRUZ,25
11934,Valencia41,REP,CARLY FIORINA,0


In [26]:
primary_data["party"].value_counts(dropna=False)

party
REP    8952
DEM    2984
Name: count, dtype: int64

In [47]:
primary_data["candidate"].unique()
candidate_party_map = (
    primary_data.dropna(subset=["candidate", "party"])
                .set_index("candidate")["party"]
                .to_dict()
)
print(candidate_party_map)

{'HILLARY RODHAM CLINTON': 'DEM', 'BERNARD SANDERS': 'DEM', 'DONALD J TRUMP': 'REP', 'JEB BUSH': 'REP', 'BENJAMIN S CARSON': 'REP', 'TED CRUZ': 'REP', 'CARLY FIORINA': 'REP', 'JOHN R KASICH': 'REP'}


In [53]:
primary_data.loc[:,"candidate_column"] = (
    "pri_" +
    primary_data["party"].str.lower() + "_" +
    primary_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
primary_result = primary_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

primary_result

candidate_column,precinct,pri_dem_CLINTON,pri_dem_SANDERS,pri_rep_BUSH,pri_rep_CARSON,pri_rep_CRUZ,pri_rep_FIORINA,pri_rep_KASICH,pri_rep_TRUMP
0,Bernalillo1,60,93,3,6,15,0,9,64
1,Bernalillo10,65,65,1,5,9,2,7,35
2,Bernalillo101,35,186,2,1,3,0,2,10
3,Bernalillo102,27,105,0,0,2,1,1,6
4,Bernalillo103,91,96,1,1,5,0,1,15
...,...,...,...,...,...,...,...,...,...
1487,Valencia5,111,111,6,5,26,1,7,127
1488,Valencia6,105,111,3,4,24,4,10,87
1489,Valencia7,142,123,5,5,26,3,10,132
1490,Valencia8,130,144,5,9,32,3,12,106


In [49]:
# Process general files
gendf_list = []

for file in general_files:
    try:
        df = pd.read_csv(file)

        # Drop rows where 'precinct' is NaN
        if 'precinct' in df.columns:
            df = df[df["precinct"].notna()]

        if 'office' in df.columns:
            df = df[df["office"] == "President"]

        gendf_list.append(df)

    except Exception as e:
        print(f"Error in {file}: {e}")

# Combine all cleaned files
gen_combined_df = pd.concat(gendf_list, ignore_index=True)
# Create a new column "precinct" by combining county and precinct
gen_combined_df["precinct"] = gen_combined_df["county"].astype(str) + gen_combined_df["precinct"].astype(str)
gen_combined_df


,county,precinct,office,district,party,candidate,votes
0,Bernalillo,Bernalillo1,President,NaN,Libertarian,GARY JOHNSON,61
1,Bernalillo,Bernalillo1,President,NaN,Constitution,DARRELL CASTLE,0
2,Bernalillo,Bernalillo1,President,NaN,Republican,DONALD J TRUMP,311
3,Bernalillo,Bernalillo1,President,NaN,Democratic,HILLARY RODHAM CLINTON,346
4,Bernalillo,Bernalillo1,President,NaN,Green,JILL STEIN,9
...,...,...,...,...,...,...,...
11931,Valencia,Valencia41,President,NaN,Democratic,HILLARY RODHAM CLINTON,379
11932,Valencia,Valencia41,President,NaN,Green,JILL STEIN,6
11933,Valencia,Valencia41,President,NaN,NaN,GLORIA LA RIVA,0
11934,Valencia,Valencia41,President,NaN,NaN,EVAN MCMULLIN,17


In [77]:
general_data = gen_combined_df[["precinct", "party", "candidate", "votes"]]
# general_data = general_data[~general_data["candidate"].isin(["YES", "NO"])]
general_data.loc[:,"party"] = general_data.apply(
    lambda row: candidate_party_map.get(row["candidate"], row["party"]) if pd.isna(row["party"]) else row["party"],
    axis=1
)
general_data.loc[:, "party"] = general_data["party"].replace({
    "Democratic": "DEM",
    "Republican": "REP"
})
general_data = general_data[general_data["party"].isin(["DEM", "REP"])] # Analyzing only republican and democratic

general_data

,precinct,party,candidate,votes
2,Bernalillo1,REP,DONALD J TRUMP,311
3,Bernalillo1,DEM,HILLARY RODHAM CLINTON,346
10,Bernalillo2,REP,DONALD J TRUMP,270
11,Bernalillo2,DEM,HILLARY RODHAM CLINTON,288
18,Bernalillo3,REP,DONALD J TRUMP,352
...,...,...,...,...
11915,Valencia39,DEM,HILLARY RODHAM CLINTON,518
11922,Valencia40,REP,DONALD J TRUMP,387
11923,Valencia40,DEM,HILLARY RODHAM CLINTON,335
11930,Valencia41,REP,DONALD J TRUMP,495


In [78]:
general_data["party"].value_counts(dropna=False)


party
REP    1492
DEM    1492
Name: count, dtype: int64

In [79]:
general_data["candidate_column"] = (
    "gen_" +
    general_data["party"].str.lower() + "_" +
    general_data["candidate"].str.split().str[-1].str.upper()
)

# pivot the table
general_result = general_data.pivot_table(
    index="precinct",
    columns="candidate_column",
    values="votes",
    aggfunc="sum",  
    fill_value=0
).reset_index()

general_result

candidate_column,precinct,gen_dem_CLINTON,gen_rep_TRUMP
0,Bernalillo1,346,311
1,Bernalillo10,287,223
2,Bernalillo101,341,57
3,Bernalillo102,231,35
4,Bernalillo103,490,125
...,...,...,...
1487,Valencia5,360,502
1488,Valencia6,404,472
1489,Valencia7,483,591
1490,Valencia8,507,716


In [80]:
combined = pd.merge(result, general_result, on="precinct", how="inner")
combined["rep_primary_total"] = combined.filter(like="pri_rep_").sum(axis=1)
combined["dem_primary_total"] = combined.filter(like="pri_dem_").sum(axis=1)
combined["general_total"] = combined.filter(like="gen_").sum(axis=1)
combined

candidate_column,precinct,pri_dem_CLINTON,pri_dem_SANDERS,pri_rep_BUSH,pri_rep_CARSON,pri_rep_CRUZ,pri_rep_FIORINA,pri_rep_KASICH,pri_rep_TRUMP,gen_dem_CLINTON,gen_rep_TRUMP,rep_primary_total,dem_primary_total,general_total
0,Bernalillo1,60,93,3,6,15,0,9,64,346,311,97,153,657
1,Bernalillo10,65,65,1,5,9,2,7,35,287,223,59,130,510
2,Bernalillo101,35,186,2,1,3,0,2,10,341,57,18,221,398
3,Bernalillo102,27,105,0,0,2,1,1,6,231,35,10,132,266
4,Bernalillo103,91,96,1,1,5,0,1,15,490,125,23,187,615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487,Valencia5,111,111,6,5,26,1,7,127,360,502,172,222,862
1488,Valencia6,105,111,3,4,24,4,10,87,404,472,132,216,876
1489,Valencia7,142,123,5,5,26,3,10,132,483,591,181,265,1074
1490,Valencia8,130,144,5,9,32,3,12,106,507,716,167,274,1223


In [74]:
combined.to_csv("NM.csv", index=False)
